In [ ]:
import os
import fitz  # PyMuPDF
from PyPDF2 import PdfReader, PdfWriter
from PIL import Image
from io import BytesIO

# Function to check if a PDF has only one page
def has_one_page(pdf_path):
    doc = fitz.open(pdf_path)
    return doc.page_count == 1

# Function to determine if a PDF is in landscape or portrait mode
def is_landscape(page):
    return page.rect.width > page.rect.height

# Function to extract the bottom part of a PDF
def extract_bottom_part(pdf_path, output_path):
    # Open the PDF
    pdf_document = fitz.open(pdf_path)

    # Assuming the PDF has only one page
    page = pdf_document[0]
    rect = page.rect

    # Determine if the page is in landscape or portrait
    if is_landscape(page):
        # Define the bottom-right quadrant for landscape
        mid_x = rect.width / 2
        mid_y = rect.height / 2
        bottom_right = fitz.Rect(mid_x, mid_y, rect.width, rect.height)
        pix = page.get_pixmap(clip=bottom_right)
    else:
        # Define the bottom half for portrait
        mid_y = rect.height / 2
        bottom_part = fitz.Rect(0, mid_y, rect.width, rect.height)
        pix = page.get_pixmap(clip=bottom_part)

    # Convert pixmap to image
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Save image to a BytesIO buffer
    img_buffer = BytesIO()
    img.save(img_buffer, format="PNG")
    img_buffer.seek(0)

    # Create a new PDF with the image as a single page
    img_pdf = fitz.open()  # Create a new PDF
    img_page = img_pdf.new_page(width=pix.width, height=pix.height)  # Create a new page
    img_page.insert_image(img_page.rect, stream=img_buffer.getvalue())  # Insert the image

    # Save the temporary PDF to a buffer
    img_pdf_buffer = BytesIO()
    img_pdf.save(img_pdf_buffer)
    img_pdf.close()

    # Read the temporary PDF buffer with PdfReader
    img_pdf_buffer.seek(0)
    new_pdf = PdfWriter()
    cropped_pdf_reader = PdfReader(img_pdf_buffer)
    new_pdf.add_page(cropped_pdf_reader.pages[0])

    # Save the new PDF
    with open(output_path, 'wb') as f:
        new_pdf.write(f)

    pdf_document.close()

# Function to loop through folders and process PDFs
def process_pdfs_in_folders(root_folder, output_folder):
    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.pdf'):
                pdf_path = os.path.join(foldername, filename)
                if has_one_page(pdf_path):
                    # Create output directory if it doesn't exist
                    os.makedirs(output_folder, exist_ok=True)

                    # Define output path
                    output_path = os.path.join(output_folder, filename)

                    # Extract the bottom part and save as a new PDF
                    extract_bottom_part(pdf_path, output_path)
                    print(f"Processed: {pdf_path} -> {output_path}")

# Define the root folder containing all PDFs and the output folder
root_folder = 'FILE LOCATION'
output_folder = 'FILE LOCATION'

# Process the PDFs
process_pdfs_in_folders(root_folder, output_folder)
